# Chapter 04-01: tf.data API — 효율적인 데이터 파이프라인

## 학습 목표
- tf.data.Dataset의 다양한 생성 방법을 이해한다
- map/filter/batch/shuffle/prefetch의 역할과 올바른 순서를 안다
- AUTOTUNE으로 자동 최적화한다

## 목차
1. Dataset 생성
2. 변환 연산
3. 파이프라인 최적화
4. 이미지 디렉토리 로드

---
### 🐣 초등학생을 위한 tf.data 친절 설명!

#### 🍽️ 데이터 파이프라인이 왜 필요해요?

> 💡 **비유**: 식당 주방처럼!
> - 손님(GPU)이 밥(데이터)을 먹는 속도가 매우 빨라요
> - 재료를 하나씩 손질하면(느린 데이터 로딩) GPU가 기다려요! 낭비!
> - **파이프라인** = 미리 재료 손질 + 대기열 준비 → GPU가 쉬지 않아요!

#### 🔄 tf.data 핵심 변환 연산

| 연산 | 역할 | 비유 |
|------|------|------|
| `.map()` | 각 데이터에 함수 적용 | 재료 손질 🔪 |
| `.shuffle()` | 데이터 무작위 섞기 | 카드 섞기 🃏 |
| `.batch()` | N개씩 묶어 배치 생성 | 접시에 담기 🍽️ |
| `.cache()` | 첫 에포크 후 메모리 저장 | 자주 쓰는 것 서랍에 |
| `.prefetch()` | 다음 배치 미리 준비 | 다음 손님 음식 미리 조리 🍳 |

#### ⚡ 최적 파이프라인 순서 (권장)

```
from_tensor_slices()
    → .map(전처리)      ← CPU 병렬 처리
    → .cache()          ← 첫 에포크 후 메모리에 저장
    → .shuffle(buffer)  ← 랜덤 셔플
    → .batch(N)         ← 배치 생성
    → .prefetch(AUTO)   ← GPU 학습 중 CPU가 다음 배치 준비
```

> 💡 **shuffle의 buffer_size**: 클수록 더 잘 섞이지만 메모리 많이 사용!
> 보통 전체 데이터셋 크기 정도로 설정해요.


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time

# 한글 폰트 설정 (macOS)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

print('TensorFlow 버전:', tf.__version__)
print('NumPy 버전:', np.__version__)

## 수학적 기초

**Min-Max 정규화**
$$x' = \frac{x - x_{\min}}{x_{\max} - x_{\min}}$$

**Z-score 표준화**
$$x' = \frac{x - \mu}{\sigma}$$

파이프라인 내 `.map()`에서 이 수식을 적용해 픽셀값 [0,255]을 [0,1]로 변환한다.

## 1. Dataset 생성

In [ ]:
# -----------------------------------------------------------
# 방법 1: from_tensor_slices — numpy 배열 또는 텐서에서 생성
# -----------------------------------------------------------

# MNIST 데이터 로드
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print(f'훈련 데이터 shape: {x_train.shape}, 레이블 shape: {y_train.shape}')

# numpy 배열을 Dataset으로 변환
# 각 슬라이스는 (이미지, 레이블) 쌍
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
print(f'Dataset 원소 spec: {train_dataset.element_spec}')

# 첫 번째 배치 확인
for img, label in train_dataset.take(1):
    print(f'이미지 shape: {img.shape}, dtype: {img.dtype}')
    print(f'레이블: {label.numpy()}')

In [ ]:
# -----------------------------------------------------------
# 방법 2: from_generator — 파이썬 제너레이터에서 생성
# 메모리에 모든 데이터를 올리지 않고 동적으로 생성할 때 유용
# -----------------------------------------------------------

def data_generator():
    """간단한 수열 제너레이터: (x, x^2) 쌍을 생성한다."""
    for i in range(10):
        yield (float(i), float(i ** 2))

gen_dataset = tf.data.Dataset.from_generator(
    data_generator,
    output_signature=(
        tf.TensorSpec(shape=(), dtype=tf.float32),  # x
        tf.TensorSpec(shape=(), dtype=tf.float32),  # x^2
    )
)

print('제너레이터 Dataset 원소:')
for x, x_sq in gen_dataset:
    print(f'  x={x.numpy():.0f}, x²={x_sq.numpy():.0f}')

In [ ]:
# -----------------------------------------------------------
# 방법 3: tf.data.Dataset.range — 정수 범위 Dataset
# 디버깅이나 인덱스 기반 파이프라인에서 편리하다
# -----------------------------------------------------------

range_ds = tf.data.Dataset.range(5)
print('range Dataset:', list(range_ds.as_numpy_iterator()))

# 방법 4: from_tensors — 전체를 단일 텐서로 감싸기 (from_tensor_slices와 차이)
# from_tensors: Dataset에 원소가 1개 (배열 전체)
# from_tensor_slices: Dataset에 원소가 N개 (각 행)
ds_single = tf.data.Dataset.from_tensors([1, 2, 3])
ds_sliced = tf.data.Dataset.from_tensor_slices([1, 2, 3])

print('from_tensors 원소 수:', sum(1 for _ in ds_single))    # 1
print('from_tensor_slices 원소 수:', sum(1 for _ in ds_sliced))  # 3

## 2. 변환 연산 — map, filter, batch, shuffle, prefetch

In [ ]:
# -----------------------------------------------------------
# .map() — 각 원소에 함수를 적용한다
# num_parallel_calls=AUTOTUNE으로 CPU 코어 수에 맞게 자동 병렬화
# -----------------------------------------------------------

AUTOTUNE = tf.data.AUTOTUNE

def preprocess(image, label):
    """Min-Max 정규화: [0,255] → [0.0, 1.0]"""
    image = tf.cast(image, tf.float32) / 255.0
    # 채널 차원 추가: (28,28) → (28,28,1)
    image = tf.expand_dims(image, axis=-1)
    return image, label

mapped_ds = train_dataset.map(preprocess, num_parallel_calls=AUTOTUNE)

for img, lbl in mapped_ds.take(1):
    print(f'.map() 후 — shape: {img.shape}, min: {img.numpy().min():.3f}, max: {img.numpy().max():.3f}')

In [ ]:
# -----------------------------------------------------------
# .filter() — 조건을 만족하는 원소만 남긴다
# 예: 레이블이 0~4인 클래스만 선택 (MNIST 절반)
# -----------------------------------------------------------

filtered_ds = mapped_ds.filter(lambda img, lbl: lbl < 5)

# 필터 전후 원소 수 비교
count_before = sum(1 for _ in mapped_ds)
count_after  = sum(1 for _ in filtered_ds)
print(f'.filter() 전: {count_before}개, 후: {count_after}개')
print(f'비율: {count_after/count_before:.1%}  (클래스 0-4이므로 약 50%)')

In [ ]:
# -----------------------------------------------------------
# .shuffle() — 데이터를 무작위로 섞는다
# buffer_size: 임시로 메모리에 올려 셔플할 원소 수
#   - 너무 작으면 셔플 효과가 약함
#   - 데이터셋 전체 크기와 같으면 완전 셔플 (메모리 주의)
# -----------------------------------------------------------

BUFFER_SIZE = 10_000  # 메모리와 셔플 효과의 균형점
SEED = 42

shuffled_ds = mapped_ds.shuffle(buffer_size=BUFFER_SIZE, seed=SEED)

# 셔플 효과 확인: 첫 10개 레이블 출력
labels_before = [lbl.numpy() for _, lbl in mapped_ds.take(10)]
labels_after  = [lbl.numpy() for _, lbl in shuffled_ds.take(10)]
print(f'셔플 전 첫 10 레이블: {labels_before}')
print(f'셔플 후 첫 10 레이블: {labels_after}')

In [ ]:
# -----------------------------------------------------------
# .batch() — 원소를 N개씩 묶어 배치를 만든다
# drop_remainder=True: 마지막 불완전 배치 제거 (모델 입력 shape 고정 시 유용)
# -----------------------------------------------------------

BATCH_SIZE = 32

batched_ds = shuffled_ds.batch(BATCH_SIZE, drop_remainder=True)

for imgs, lbls in batched_ds.take(1):
    print(f'.batch() 후 — 이미지 shape: {imgs.shape}, 레이블 shape: {lbls.shape}')
    # (32, 28, 28, 1), (32,)

In [ ]:
# -----------------------------------------------------------
# .prefetch() — 현재 배치를 GPU로 보내는 동안
#               CPU는 미리 다음 배치를 준비한다 (비동기 처리)
# AUTOTUNE: 시스템 환경에 맞게 프리패치 버퍼 크기 자동 결정
# -----------------------------------------------------------

prefetched_ds = batched_ds.prefetch(buffer_size=AUTOTUNE)

print('최종 파이프라인 spec:')
print(prefetched_ds.element_spec)

# 배치 수 확인
num_batches = sum(1 for _ in prefetched_ds)
print(f'총 배치 수: {num_batches}  (60000 // 32 = {60000 // 32})')

## 3. 파이프라인 최적화

### 올바른 파이프라인 순서
```
dataset
  .cache()        # 디스크/메모리 캐시 (shuffle 전)
  .shuffle()      # 셔플
  .batch()        # 배치
  .map()          # 변환 (배치 단위)
  .prefetch(AUTOTUNE)  # 비동기 프리패치
```

In [ ]:
# -----------------------------------------------------------
# 최적화된 파이프라인 — 권장 순서로 조립
# -----------------------------------------------------------

def build_pipeline(x, y, batch_size=32, buffer_size=10_000, training=True):
    """
    x: numpy 이미지 배열
    y: numpy 레이블 배열
    training: True이면 셔플 포함
    """
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(preprocess, num_parallel_calls=AUTOTUNE)  # 개별 전처리
    ds = ds.cache()                                        # 메모리 캐시
    if training:
        ds = ds.shuffle(buffer_size=buffer_size, seed=42)  # 셔플 (훈련 시)
    ds = ds.batch(batch_size, drop_remainder=training)     # 배치
    ds = ds.prefetch(buffer_size=AUTOTUNE)                 # 비동기 프리패치
    return ds

train_ds = build_pipeline(x_train, y_train, training=True)
test_ds  = build_pipeline(x_test,  y_test,  training=False)

print('훈련 pipeline spec:', train_ds.element_spec)
print('테스트 pipeline spec:', test_ds.element_spec)

In [ ]:
# -----------------------------------------------------------
# prefetch 유무에 따른 배치 로드 시간 비교
# -----------------------------------------------------------

def benchmark(dataset, num_epochs=2):
    """dataset을 num_epochs 순회하는 시간을 측정한다."""
    start = time.perf_counter()
    for _ in range(num_epochs):
        for _ in dataset:
            pass  # 실제 학습 대신 로드만 측정
    elapsed = time.perf_counter() - start
    return elapsed

# prefetch 없는 파이프라인
ds_no_prefetch = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .cache()
    .shuffle(10_000)
    .batch(32)
)

# prefetch 있는 파이프라인
ds_with_prefetch = ds_no_prefetch.prefetch(AUTOTUNE)

t_no  = benchmark(ds_no_prefetch)
t_yes = benchmark(ds_with_prefetch)

print(f'prefetch 없음: {t_no:.3f}초')
print(f'prefetch 있음: {t_yes:.3f}초')
print(f'속도 향상: {t_no/t_yes:.2f}x')

## 4. 이미지 디렉토리 로드

In [ ]:
# -----------------------------------------------------------
# tf.keras.utils.image_dataset_from_directory
# 디렉토리 구조:
#   data/
#     cats/  ← 클래스명이 레이블이 된다
#       001.jpg
#     dogs/
#       001.jpg
# -----------------------------------------------------------

import os
import pathlib

# 실습용 더미 디렉토리 구조 생성
dummy_dir = pathlib.Path('/tmp/sample_images')
for cls in ['cats', 'dogs']:
    (dummy_dir / cls).mkdir(parents=True, exist_ok=True)

# 더미 이미지(노이즈) 저장
import cv2 as _cv2_check
try:
    import cv2
    for cls_idx, cls in enumerate(['cats', 'dogs']):
        for i in range(5):
            img = np.random.randint(0, 256, (64, 64, 3), dtype=np.uint8)
            cv2.imwrite(str(dummy_dir / cls / f'{i:03d}.jpg'), img)
    print('OpenCV로 더미 이미지 생성 완료')
except ImportError:
    # OpenCV가 없으면 matplotlib으로 저장
    for cls_idx, cls in enumerate(['cats', 'dogs']):
        for i in range(5):
            img = np.random.rand(64, 64, 3)
            plt.imsave(str(dummy_dir / cls / f'{i:03d}.jpg'), img)
    print('matplotlib으로 더미 이미지 생성 완료')

# image_dataset_from_directory 사용
image_ds = tf.keras.utils.image_dataset_from_directory(
    str(dummy_dir),
    labels='inferred',         # 디렉토리명을 레이블로 자동 추론
    label_mode='int',          # 정수 레이블 (binary: 0/1, categorical: one-hot)
    image_size=(64, 64),       # 이미지 리사이즈 크기
    batch_size=4,
    shuffle=True,
    seed=42,
)

print('클래스 이름:', image_ds.class_names)
print('Dataset spec:', image_ds.element_spec)

# 배치 시각화
for imgs, labels in image_ds.take(1):
    fig, axes = plt.subplots(1, 4, figsize=(12, 3))
    class_names = image_ds.class_names
    for ax, img, lbl in zip(axes, imgs, labels):
        ax.imshow(img.numpy().astype('uint8'))
        ax.set_title(class_names[lbl.numpy()])
        ax.axis('off')
    plt.suptitle('image_dataset_from_directory 로드 결과')
    plt.tight_layout()
    plt.show()

## 정리

| 연산 | 역할 |
|------|------|
| `.cache()` | 첫 에포크 후 데이터 메모리 저장 |
| `.shuffle(buffer)` | buffer 크기만큼 랜덤 셔플 |
| `.batch(N)` | N개씩 묶어 배치 생성 |
| `.map(fn)` | 각 샘플에 함수 적용 (병렬 가능) |
| `.prefetch(AUTOTUNE)` | 모델 학습 중 다음 배치 미리 준비 |

**다음**: 02_data_augmentation.ipynb — 데이터 증강